In [1]:
## Goal of this script is to fit a Bingham distribution to the data Voxelwise and Filter for a 95% Intervall. 

In [1]:
from plyfile import PlyData, PlyElement
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
## Load data - in world space. Find all 

with open('/home/xderes/Downloads/904044/cst_ukf.ply', 'rb') as f:
    plydata = PlyData.read(f)
    num_verts = plydata['vertices'].count
    num_fiber = plydata['fiber'].count
    vertices = np.zeros((num_verts, 4))
    endindex = np.zeros((num_fiber + 1), dtype=int)
    vertices[:, 0] = plydata['vertices'].data['x']
    vertices[:, 1] = plydata['vertices'].data['y']
    vertices[:, 2] = plydata['vertices'].data['z']
    vertices[:, 3] = plydata['vertices'].data['seedpoint']
    endindex[1:] = plydata['fiber'].data['endindex']

In [3]:
# get dirs to neighbor in each point and norm direction to 1
data = np.zeros((len(vertices), 10))
data[:,:4] = vertices
for i,x in enumerate(endindex[1:]):
    before = endindex[i]
    data[before:x-1, 4:7] = data[before: x - 1, :3] - data[before + 1: x:, :3]
    data[before:x-1, 4:7] = np.divide(data[before:x-1, 4:7], np.linalg.norm(data[before:x-1, 4:7], axis=1)[..., np.newaxis], where=(np.sum(data[before:x-1, 4:7], axis=1)!=0)[...,np.newaxis])
    data[before + 1:x, 7:] = data[before + 1: x, :3] - data[before: x -1:, :3]
    data[before + 1:x, 7:] = np.divide(data[before + 1:x, 7:], np.linalg.norm(data[before + 1:x, 7:], axis=1)[..., np.newaxis], where=(np.sum(data[before + 1:x, 7:], axis=1)!=0)[...,np.newaxis])


In [4]:
# select points in a given radius

In [5]:
selected_points = data[np.linalg.norm(data[:, :3] - np.array((1.77251969e+01, -1.44250541e+01,  1.30592768e-03)), axis=1) < 1]

In [6]:
dirs = np.vstack((selected_points[:,4:7], selected_points[:,7:]))

In [7]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u,v = np.mgrid[0:2*np.pi:30j, 0:np.pi:30j]
x = np.sin(v) * np.sin(u)
y = np.sin(v) * np.cos(u)
z = np.cos(v)
ax.plot_surface(x,y,z, alpha=0.5)
ax.scatter3D(dirs[:,0], dirs[:, 1], dirs[:,2])
plt.show()

In [8]:
#get mean direction

first = np.random.randint(0, len(dirs) + 1)
center = [[dirs[first], -dirs[first]]]
while True:
    first = np.linalg.norm(dirs - center[-1][0], axis=1) 
    second = np.linalg.norm(dirs - center[-1][1], axis=1)
    select = first - second <= 0
    new_center = np.mean(dirs[select], axis = 0)
    new_center /= np.linalg.norm(new_center)
    center.append([new_center, -new_center])
    print(center[-1][0])
    if np.linalg.norm(center[-1][0] - center[-2][0]) < 0.001:
        break

[-0.31292304 -0.04433635 -0.94874309]
[-0.31292304 -0.04433635 -0.94874309]


In [9]:
a = np.array(center)[:,0,:]
a *= 1.1

In [11]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u,v = np.mgrid[0:2*np.pi:30j, 0:np.pi:30j]
x = np.sin(v) * np.sin(u)
y = np.sin(v) * np.cos(u)
z = np.cos(v)
ax.plot_surface(x,y,z, alpha=0.5)
ax.scatter3D(dirs[select,0], dirs[select, 1], dirs[select,2], alpha=1)
ax.scatter3D(a[:,0], a[:, 1], a[:,2], c='r', alpha=1)
plt.show()

In [12]:
# Now following:https://arxiv.org/pdf/1506.08105.pdf

mean_dir = np.array(center)[-1,0,:]
selected_dirs = dirs[select]

In [13]:
S = selected_dirs.T @ selected_dirs/len(selected_dirs)

In [14]:
# Compute rotation matrix H
north_pole = np.array([1,0,0])
v = np.cross(mean_dir, north_pole)
s = np.linalg.norm(v)
c = np.dot(mean_dir, north_pole)
vx = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]]) 
H = np.eye(3) + vx + np.dot(vx,vx)* (1-c)/(s**2) 

In [15]:
# Compute dispersion matrix B 
B = H.T @ S @ H
B_E, B_L = np.linalg.eig(B)
psi = np.arctan(2*B_L[1,2]/(B_L[1,1] - B_L[2,2]))/2

In [18]:
#dirs_rot = R @ dirs.T
K = np.array([[1,0,0], [0, np.cos(psi), np.sin(psi)], [0, -np.sin(psi), np.cos(psi)]])


In [19]:
dirs_rot =   K @ H    @ dirs.T
a_rot = (H @ a.T).T

In [20]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u,v = np.mgrid[0:2*np.pi:30j, 0:np.pi:30j]
x = np.sin(v) * np.sin(u)
y = np.sin(v) * np.cos(u)
z = np.cos(v)
ax.plot_surface(x,y,z, alpha=0.5)
ax.scatter3D(dirs_rot.T[select,0], dirs_rot.T[select, 1], dirs_rot.T[select,2], alpha=1)
ax.scatter3D(a_rot[:,0], a_rot[:, 1], a_rot[:,2], c='r', alpha=1)
origin0 = [0,0,0]
origin1 = [1,0,0]
p0 = [1,0,0]
p1 = [0,1,0]
p2 = [0,0,1]
X, Y, Z = zip(origin0,origin1,origin1) 
U, V, W = zip(p0,p1,p2)
ax.quiver(X,Y,Z,U,V,W,arrow_length_ratio=0.01)
plt.show()

In [21]:
# kappa, beta estimate: 

r1 = np.linalg.norm(np.mean(dirs_rot.T[select], axis=0))
r2 = B_E[0] - B_E[1]
kappa_guess = 1/(2 -2*r1 -r2) + 1/(2-2*r1 + r2)
beta_guess = 0.5*(1/(2 -2*r1 -r2) - 1/(2-2*r1 + r2))

In [28]:
def make_sphere(r_1=5000, r_2=5000):
     phi = np.linspace(0,np.pi , r_1)
     theta = np.linspace(0, 2*np.pi  , r_2)
     phi, theta = np.meshgrid(phi, theta)
     phi = phi.flatten() 
     theta = theta.flatten()
     return theta, phi
 
theta, phi = make_sphere(2000, 2000)

from scipy.special import sph_harm
def normalize_constant(k,b,theta, phi):
     return 4*np.pi*np.sum(np.exp((np.cos(phi)**2*k)[..., np.newaxis]+b[np.newaxis]*(np.sin(theta)**2*np.sin(phi)**2)[...,np.newaxis])*np.real(sph_harm(    0,0, theta, phi))[:,np.newaxis]*np.sin(phi)[..., np.newaxis], axis=0) /len(phi)

from pybingham import bingham_F
from numpy import exp
def log_likelihood(v, x,n):
    b_c = n*np.log(normalize_constant(exp(v[0]), np.array([exp(v[1])]), theta, phi))
    gamma1, gamma2 = rot(*v[2:])
    return b_c - (exp(v[0])*gamma1 @ x @ gamma1[:, np.newaxis] + exp(v[1])*gamma2 @ x @ gamma2[:, np.newaxis])[0]

In [29]:
def rot(alpha, psi, eta):
    z1 = np.array([np.cos(alpha), np.sin(alpha)* np.cos(eta), np.sin(alpha) * np.sin(eta)])
    z2 = np.array([- np.cos(psi)*np.sin(alpha), np.cos(psi)*np.cos(alpha)*np.cos(eta) - np.sin(psi)*np.sin(eta), np.cos(psi)*np.cos(alpha)*np.sin(eta) + np.sin(psi) *np.cos(eta)])
    #z3 = np.array([np.sin(psi)* np.sin(alpha), - np.sin(psi)*np.cos(alpha)*np.cos(eta) - np.cos(psi)*np.sin(eta), -np.sin(psi)*np.cos(alpha)*np.sin(psi) + np.cos(psi)*np.cos(eta)]) 
    return z1, z2

In [31]:
from scipy.optimize import minimize

In [32]:
x = dirs_rot @ dirs_rot.T
v = [3, 3, 0,0,0]

In [33]:
ret = minimize(log_likelihood, v, args=(x, dirs_rot.shape[1]), method = 'Nelder-Mead')

In [36]:
def bingham(points, x):
    kappa, beta = np.exp(x[:2])
    gamma1, gamma2 = rot(*x[2:])
    return np.exp(kappa * (points @ gamma1)**2  + (beta) * (points @ gamma2)**2)
        

In [69]:
%matplotlib qt
from matplotlib import cm
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u,v = np.mgrid[0:2*np.pi:200j, 0:np.pi:100j]
x = np.sin(v) * np.sin(u)
y = np.sin(v) * np.cos(u)
z = np.cos(v)
ax.plot_surface(x,y,z, alpha=1, facecolors=cm.jet(bingham(np.array([x.flatten(),y.flatten(),z.flatten()]).T, ret.x).reshape((200,100))/np.max(bingham(np.array([x,y,z]).T, ret.x))))
ax.scatter3D(dirs_rot.T[:,0], dirs_rot.T[:, 1], dirs_rot.T[:,2], alpha=0.1)
ax.scatter3D(a_rot[:,0], a_rot[:, 1], a_rot[:,2], c='r', alpha=1)
origin0 = [0,0,0]
origin1 = [1,0,0]
p0 = [1,0,0]
p1 = [0,1,0]
p2 = [0,0,1]
X, Y, Z = zip(origin0,origin1,origin1) 
U, V, W = zip(p0,p1,p2)
ax.quiver(X,Y,Z,U,V,W,arrow_length_ratio=0.1)
plt.show()

In [94]:
%matplotlib qt
from matplotlib import cm
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u,v = np.mgrid[0:2*np.pi:200j, 0:np.pi:100j]
x = np.sin(v) * np.sin(u)
y = np.sin(v) * np.cos(u)
z = np.cos(v)
p_dirs = (H.T @ K.T @ np.array((x.flatten(),y.flatten(),z.flatten()))).reshape(3,200,100)
ax.plot_surface(p_dirs[0], p_dirs[1], p_dirs[2], alpha=1, facecolors=cm.jet(bingham(np.array([x.flatten(),y.flatten(),z.flatten()]).T, ret.x).reshape((200,100))/np.max(bingham(np.array([x,y,z]).T, ret.x))))
ax.scatter3D(dirs[:,0]+0.01, dirs[:, 1]+0.01, dirs[:,2]+0.01, alpha=0.1)
ax.scatter3D(a_rot[:,0], a_rot[:, 1], a_rot[:,2], c='r', alpha=1)
origin0 = [0,0,0]
origin1 = [0,0,1]
p0 = [1,0,0]
p1 = [0,1,0]
p2 = [0,0,1]
X, Y, Z = zip(origin0,origin1,origin1) 
U, V, W = zip(p0,p1,p2)
ax.quiver(X,Y,Z,U,V,W,arrow_length_ratio=0.1)
plt.show()

In [50]:
a = bingham(p_dirs, ret.x)

In [61]:
np.rollaxis((K @ H  @ np.rollaxis(np.array([x,y,z]), axis=1)), axis=1).shape

(3, 200, 100)

In [84]:
a = np.array((x,y,z))

In [88]:
p_dirs = (K @ H @ np.array((x.flatten(),y.flatten(),z.flatten())))

In [91]:
np.linalg.norm(p_dirs, axis=0)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [7]:
import numpy as np
a = np.random.random(3)
a /= np.linalg.norm(a)
b = np.random.random(3)
b /= np.linalg.norm(b)

In [18]:
c = np.cross(a,b)
c /= np.linalg.norm(c)

In [20]:
d = np.cross(a, c)
d /= np.linalg.norm(d)

In [21]:
e = np.cross(b,c)
e /= np.linalg.norm(e)

In [71]:
A = np.array((a,c, d)).T
B = np.array((b,c,e)).T

In [72]:
R =  B @ np.linalg.inv(A)

In [73]:
R @ A

array([[ 0.42332592, -0.89453954, -0.14350672],
       [ 0.66027026,  0.41308381, -0.62722002],
       [ 0.62035341,  0.17076527,  0.76550694]])

In [74]:
B

array([[ 0.42332592, -0.89453954, -0.14350672],
       [ 0.66027026,  0.41308381, -0.62722002],
       [ 0.62035341,  0.17076527,  0.76550694]])

In [75]:
b

array([0.42332592, 0.66027026, 0.62035341])

In [76]:
R @ a

array([0.42332592, 0.66027026, 0.62035341])